# Doing a Linear regression without Scikit Learn

There's nothing better to understand the gradient descent algorithm than to code it from scratch. It may seem difficult at first glance but don't worry, we will guide you step by step. It is also a fantastic occasion to practice python key concepts like **Classes**!

Don't hesitate to come back to your Machine Learning course on linear regression to refresh your memory. 

Our goal will be to code a simple linear regression such as : 

$f(x) = \beta_1 \times x + \beta_0$

* Import the following libraries: 
  * Numpy 

In [1]:
import numpy as np

* Define a `Model` class that will take two methods: 
  1. `__init__(self)`, the class builder which will allow you to define an attribute $\beta_0$ (`beta_0` in your code) and an attribute $\beta_1$ (`beta_1` in your code). These attributes represent the coefficients/parameters of the model an we will be initialize them randomly using Numpy (cf: `np.random.randn`).
  2. `__call__(self, x)`, a special method that will turn our class into a callable which will return $\beta_1 \times x + \beta_0$ when called.

In [2]:
class Model():
  def __init__(self):
    self.beta_1 = np.random.randn(1)
    self.beta_0 = np.random.randn(1)
  
  def __call__(self, x):
    return self.beta_1 * x + self.beta_0

* Create an instance of your class `Model`

In [3]:
model = Model()

In [4]:
model.beta_1

array([-1.62580741])

In [5]:
model.beta_0

array([0.89200882])

* Try doing a first "regression" by running `model(3.0)`. 
NB: If you don't have the same values as this notebook in output, this is normal since you have initialized your values randomly. 

In [6]:
model(3.0)

array([-3.98541341])

In [7]:
model.beta_1 * 3 + model.beta_0

array([-3.98541341])

* This value corresponds to a random prediction of your model. But we don't have any data yet. This time, let's use `sklearn` to import data. 
  * Import `sklearn.datasets`
  * Use the `load_diabetes()` function to load the diebetes dataset in an object called `diabetes`.
  * Print the `DESCR` attribute of the diabetes object
  * Save the content of the `data` attribute in an object named `diabetes_data`
  * Save the content of the `target` attribute in an object named `y`

In [8]:
from sklearn import datasets, linear_model

# Load the diabetes dataset
diabetes = datasets.load_diabetes()
print(diabetes.DESCR)
diabetes_data = diabetes.data
y = diabetes.target

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

* We have too much data in this dataset `diabetes_data`, take only the third column of the dataset and store it in a `diabetes_X` variable.

In [9]:
# Use only one feature
diabetes_X = diabetes_data[:,2]
diabetes_X[:5]

array([ 0.06169621, -0.05147406,  0.04445121, -0.01159501, -0.03638469])

* Visualize your data using `plotly`.

In [10]:
import plotly.express as px
fig = px.scatter(x= diabetes_X, y=y, title="Scatter plot of BMI against Diabetes metric",
                 labels={
                     "x": "BMI",
                     "y": "Diabtes Metric"
                 })
fig.show()
"""import matplotlib.pyplot as plt 
plt.scatter(diabetes_X, y)
plt.xlabel(diabetes.feature_names[2])
plt.ylabel("target")
plt.show()"""

'import matplotlib.pyplot as plt \nplt.scatter(diabetes_X, y)\nplt.xlabel(diabetes.feature_names[2])\nplt.ylabel("target")\nplt.show()'

* Now we need to define a cost function. For a linear regression, we could use MSE :

`np.mean((model(input) - y)**2)`

  * Create this function which we'll call `mse` (for mean square error). This function will take two arguments `y_pred` & `y_true`.

In [27]:
def mse(y_pred, y_true):
  # return (np.sum((y_pred - y_true)**2))/len(y_pred) alternate solution
  return np.mean((y_pred - y_true)**2) # it is important to use numpy functions
  # since they can be applied easily on numpy arrays (or array like objects)
  # and they compute very fast compared to pure python operations

* Test your function by inserting `model(diabetes_X)` & `y` as arguments. 
* Calculate the rmse as well

In [12]:
print (f" MSE : {mse(model(diabetes_X), y)}")
print (f" RMSE : {np.sqrt(mse(model(diabetes_X), y))}")

 MSE : 28810.859350311286
 RMSE : 169.7376191370413


* Visualize your regression in relation to your points 

In [13]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=diabetes_X, y=y,
                    mode='markers',
                    name='target'))
fig.add_trace(go.Scatter(x=diabetes_X, y=model(diabetes_X),
                    mode='lines',
                    name='predictions'))
fig.update_layout(
    title="Target vs Predictions",
    xaxis_title="BMI",
    yaxis_title="Diabetes metric"
    )
fig.show()

* We're going to need to compute the gradients for our variable `model.beta_1` and our constant `model.beta_0`. To do this, we're going to need to review our derivative formulas. Since we're not here to do math, we're going to give you these formulas. 
  * `derive_model_beta_1 = 2/len(y_pred)*np.sum((x @ (y_pred - y_true)))`
  * `derive_model_beta_0 = 2/len(y_pred)*(np.sum(y_pred - y_true))`

  * Feel free to read this article if you want to know more about the calculation of the derivative: [Gradient Descent Derivation](https://mccormickml.com/2014/03/04/gradient-descent-derivation/)


  * So using the above formulas, code the first function `derivative_mse_beta_1` that will take the arguments: 
    * `x` --> the values for your variable / `y_pred` --> the values predicted by your model / `y_true` --> the values of the target variable


In [14]:
# Calculate model.beta_1's derivate
def derivative_mse_beta_1(y_pred, y_true, x):
  return 2/len(y_pred)*np.sum((x @ (y_pred - y_true)))
  # return 2/len(y_pred) * np.sum(np.dot(x,(y_pred-y_true)))

* Test you function

In [15]:
derivative_mse_beta_1(model(diabetes_X), y, diabetes_X)

-4.30344374567454

* So using the above formulas, now code the `derivative_mse_beta_0` function which will take the arguments :
    * `y_pred` --> the values predicted by your model / `y_true` --> the actual values to predict

In [16]:
# Calculate model.b's derivate
def derivative_mse_beta_0(y_pred, y_true):
  return 2/len(y_pred)*(np.sum(y_pred - y_true))

* Test you function

In [17]:
derivative_mse_beta_0(model(diabetes_X), y)

-302.4829506790338

* We will try to see if we can minimize our cost function using the two gradients above. To update our variables, we need to subtract their respective gradients. Ex: 
  * `param = param - learning_rate * gradient`

  * Set a `learning_rate` to 0.1
  * Try to apply your formula on `model.beta_1` and `model.beta_0`.

In [18]:
lr = 0.1

print("OLD model.a = {}".format(model.beta_1))
print("OLD model.b = {}".format(model.beta_0))

model.beta_1 -= lr * derivative_mse_beta_1(model(diabetes_X), y, diabetes_X)
# model.a = model.a - lr * derivative_mse_a(model(diabetes_X), y, diabetes_X)
model.beta_0 -= lr * derivative_mse_beta_0(model(diabetes_X), y)

print("NEW model.a = {}".format(model.beta_1))
print("NEW model.b = {}".format(model.beta_0))

OLD model.a = [-1.62580741]
OLD model.b = [0.89200882]
NEW model.a = [-1.19546304]
NEW model.b = [31.14030389]


Wesee that the values of the two parameters have changed, let's see how it affected the predictions of the model. 
Visualize the data vs the model's predictions.

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=diabetes_X, y=y,
                    mode='markers',
                    name='target'))
fig.add_trace(go.Scatter(x=diabetes_X, y=model(diabetes_X),
                    mode='lines',
                    name='predictions'))
fig.update_layout(
    title="Target vs Predictions",
    xaxis_title="BMI",
    yaxis_title="Diabetes metric"
    )
fig.show()

We notice the predictions got a little closer to our real data
* Recalculate your MSE

In [20]:
mse(model(diabetes_X), y)

20574.37361586565

* Our MSE has dropped a lot! This is good news but the process of gradient descent is iterative. So you'll have to do it several times before arriving at accurate predictions. 
  * By making a loop, try to repeat the process from above 10,000 times. 
  * Display every 1000 epochs: mse, model.beta_1 & model.beta_0 

In [21]:
# Define learning rate and a number of iterations 
lr = 0.1
epochs = 10000

In [22]:
model = Model()
for epoch in range(epochs):
  # Calculate the loss function
  current_loss = mse(model(diabetes_X), y)
  
  # Update variables
  model.beta_1 -= lr * derivative_mse_beta_1(model(diabetes_X), y, diabetes_X)
  model.beta_0 -= lr * derivative_mse_beta_0(model(diabetes_X), y)

  # Show updated variables
  if epoch % 1000 == 0 or epoch == epochs - 1:
    print("-------------------- Epoch {} --------------------".format(epoch))
    print("Current Loss: {}".format(current_loss))
    print("beta_1 = {}".format(model.beta_1))
    print("beta_0 = {}".format(model.beta_0))

-------------------- Epoch 0 --------------------
Current Loss: 28612.542035357994
beta_1 = [0.26670379]
beta_0 = [31.64924278]
-------------------- Epoch 1000 --------------------
Current Loss: 4715.623686337922
beta_1 = [345.78480986]
beta_0 = [152.13348416]
-------------------- Epoch 2000 --------------------
Current Loss: 4224.210430038033
beta_1 = [565.52677133]
beta_0 = [152.13348416]
-------------------- Epoch 3000 --------------------
Current Loss: 4025.449399221573
beta_1 = [705.27785465]
beta_0 = [152.13348416]
-------------------- Epoch 4000 --------------------
Current Loss: 3945.0568846429396
beta_1 = [794.1564883]
beta_0 = [152.13348416]
-------------------- Epoch 5000 --------------------
Current Loss: 3912.5406697001654
beta_1 = [850.68135617]
beta_0 = [152.13348416]
-------------------- Epoch 6000 --------------------
Current Loss: 3899.388895063602
beta_1 = [886.62993634]
beta_0 = [152.13348416]
-------------------- Epoch 7000 --------------------
Current Loss: 3894.0

* Using `plotly`, view your model and actual values again

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=diabetes_X, y=y,
                    mode='markers',
                    name='target'))
fig.add_trace(go.Scatter(x=diabetes_X, y=model(diabetes_X),
                    mode='lines',
                    name='predictions'))
fig.update_layout(
    title="Target vs Predictions",
    xaxis_title="BMI",
    yaxis_title="Diabetes metric"
    )
fig.show()

**We've got a nice regression this time!** 